In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 데이터 불러오기

import pandas as pd

def csv_r(name):
    file_name = "/content/drive/MyDrive/월간공모전_유진지현/" + name + ".csv"
    df = pd.read_csv(file_name)
    df.index = df.iloc[:,0]
    df = df.iloc[:,1:]
    return df

df = pd.read_csv('/content/drive/MyDrive/월간공모전_유진지현/정리용폴더/feature_final.csv', index_col=0)
df.head()

,한국_환율_lag12,한국_선행종합지수_lag6_증가율,한국_대일본수출_lag12,일본_소비자물가지수_lag6_증가율,중국_실질실효환율_lag6_증가율,홍콩_환율_lag6_증가율,DowJones_lag6_증가율,P_Dubai_lag6_증가율,한국_수출액_증가율,한국_수출액
기간,,,,,,,,,,
2002-01,1259.0,-0.018900,1.616073e+09,-0.008166,0.070292,0.000128,0.000079,-0.041810,-0.110970,1.138188e+10
2002-02,1250.8,-0.025729,1.546805e+09,-0.007153,0.040842,0.000064,-0.127174,-0.191007,-0.211470,1.102304e+10
2002-03,1327.5,-0.015358,1.637435e+09,-0.008149,0.019790,0.000346,-0.203826,-0.315202,-0.065923,1.325132e+10
2002-04,1319.7,0.003390,1.365516e+09,-0.008149,0.020560,0.000231,-0.208922,-0.600320,0.080657,1.318453e+10
2002-05,1282.7,0.041667,1.479760e+09,-0.010263,0.017943,-0.000154,-0.057141,-0.566555,0.061117,1.417318e+10


In [33]:
train = df.loc[:"2021-12"]
test = df.loc["2022-01":]

In [34]:
target = '한국_수출액_증가율'
r_target = '한국_수출액'

features = df.drop(columns=[target, r_target]).columns

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [35]:
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [36]:
pipe = make_pipeline(
    StandardScaler(),
    lgb.LGBMRegressor(random_state=1, n_jobs=-1, learning_rate=0.2, max_depth=5, n_estimators=50,
                               num_leaves=60, reg_alpha=0))

pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('lgbmregressor',
                 LGBMRegressor(learning_rate=0.2, max_depth=5, n_estimators=50,
                               num_leaves=60, random_state=1, reg_alpha=0))])

In [39]:
df_target = df[[target, r_target]]

X_train_p = X_train.copy()
X_train_p['한국_수출액_증가율_예측치'] = pipe.predict(X_train)
X_test_p = X_test.copy()
X_test_p['한국_수출액_증가율_예측치'] = pipe.predict(X_test)
pred = pd.concat([X_train_p['한국_수출액_증가율_예측치'], X_test_p['한국_수출액_증가율_예측치']])
df_target['한국_수출액_증가율_예측치'] = pred

df_target['한국_전년도수출액'] = df_target[r_target].shift(12)
df_target['한국_수출액_예측치'] = df_target['한국_전년도수출액'] * (1+df_target['한국_수출액_증가율_예측치'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [59]:
final_pred = df_target.reset_index().iloc[240:, [0,5]]
final_pred.to_csv('final_pred.csv', index=True, encoding='utf-8-sig')

In [60]:
final_pred

,기간,한국_수출액_예측치
240,2022-01,5.390419e+10
241,2022-02,5.304186e+10
242,2022-03,5.881592e+10
243,2022-04,5.635721e+10
244,2022-05,5.670845e+10
245,2022-06,5.587562e+10
